# **Setup**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops chess

stockfish_path = '/content/drive/MyDrive/ProjectNLP/engines/stockfish-ubuntu-20.04-x86-64'
!chmod 777 '/content/drive/MyDrive/ProjectNLP/engines/stockfish-ubuntu-20.04-x86-64'

import chess.engine, chess.pgn, chess
import torch, transformers, os, pandas as pd, numpy as np, random
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sklearn.metrics import confusion_matrix, precision_recall_curve, PrecisionRecallDisplay, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score
from collections import Counter
from peft import PeftModel
from datasets import load_from_disk


Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
# Setting permission for the engine files
!chmod 777 '/content/drive/MyDrive/ProjectNLP/engines/stockfish-ubuntu-20.04-x86-64'
!chmod 777 '/content/drive/MyDrive/ProjectNLP/engines/maia-1900/lc0'
!chmod 777 '/content/drive/MyDrive/ProjectNLP/engines/maia-1100/lc0'
!chmod 777 '/content/drive/MyDrive/ProjectNLP/engines/maia-1400/lc0'
!chmod 777 '/content/drive/MyDrive/ProjectNLP/engines/maia-1600/lc0'

stockfish_path = '/content/drive/MyDrive/ProjectNLP/engines/stockfish-ubuntu-20.04-x86-64'
maia_1900_path = '/content/drive/MyDrive/ProjectNLP/engines/maia-1900/lc0'
maia_1100_path = '/content/drive/MyDrive/ProjectNLP/engines/maia-1100/lc0'
maia_1600_path = '/content/drive/MyDrive/ProjectNLP/engines/maia-1600/lc0'
maia_1400_path = '/content/drive/MyDrive/ProjectNLP/engines/maia-1400/lc0'

In [3]:
# @title Load Model
def load_with_adapters(adapters_path, quantize=False):

  torch.cuda.empty_cache()
  gc.collect()

  model_name = "Waterhorse/chessgpt-base-v1"
  tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left', output_attentions=True)


  tokenizer.pad_token = tokenizer.eos_token
  if quantize:
    nf4_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4"
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=nf4_config,
        trust_remote_code=True,
      #  torch_dtype=torch.float16
        low_cpu_mem_usage=True
    )
  else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16
    )

  model.to('cuda:0')

  if adapters_path:
    m = PeftModel.from_pretrained(model, adapters_path)
    model = m.merge_and_unload()

  torch.cuda.empty_cache()
  gc.collect()

  return model,tokenizer

### Evaulation

In [4]:
# @title Run Helpers


def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Q: {example['input'][i]}\n ### A:"
        output_texts.append(text)
    return output_texts

def generate_outputs(model, tokenizer, batch, beam_size, max_new_tokens, temp=0.7):
    # Initialize a dictionary to store outputs for each element in the batch
    batch_outputs = {i: [] for i in range(len(batch))}

    torch.cuda.empty_cache()
    gc.collect()


    inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True).to(model.device)
    input_lengths = [len(input) for input in inputs["input_ids"]]  # Individual input lengths for each sequence

    outputs = model.generate(
        **inputs, max_new_tokens=max_new_tokens,do_sample=True, temperature=temp, top_p=0.7, top_k=50,
        return_dict_in_generate=True, num_return_sequences=beam_size#, num_beams=beam_size, diversity_penalty=2.0
    )

    for i in range(len(batch)):
        # Extract the sequences for the current input
        beam_outputs = outputs.sequences[i * beam_size: (i + 1) * beam_size]
        # Trim sequences based on the input lengths and decode
        beam_outputs_trimmed = [seq[input_lengths[i]:].tolist() for seq in beam_outputs]
        decoded = tokenizer.batch_decode(beam_outputs_trimmed, skip_special_tokens=True)
        # Append decoded beams for the current input to batch_outputs
        batch_outputs[i].extend(decoded)

    torch.cuda.empty_cache()
    gc.collect()
    return batch_outputs

def evaluate_dataset(input_data, model, tokenizer, batch_size, max_new_tokens, n_examples=2, beam_size=5, label_data_in=None):
    """
    Evaluates dataset and returns a DataFrame with Input, Label, and Output for each repeat.
    """
    # Extracting a subset if n_examples is provided
    if n_examples != -1:
        input_data = input_data[:n_examples]
        if label_data_in is not None:
          label_data = label_data_in[:n_examples]

    prompts = input_data.tolist()

    # Lists to store each output
    all_outputs = [[] for _ in range(beam_size)]

    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i+batch_size]
        batch_outputs = generate_outputs(model, tokenizer, batch, beam_size, max_new_tokens)

        # Convert the dictionary back to the format we want (list of lists)
        for idx in range(beam_size):
            repeated_output_for_this_repeat = [batch_outputs[j][idx] for j in range(len(batch))]
            all_outputs[idx].extend(repeated_output_for_this_repeat)

    output = pd.DataFrame()
    output['input'] = input_data
    if label_data_in is not None:
      output['label'] = label_data_in
    for i in range(beam_size):
        output[f'outputs_{i}'] = all_outputs[i]
    print(all_outputs)
    torch.cuda.empty_cache()
    gc.collect()
    return output




In [5]:
# @title Eval Helpers
def exact_match_metric(output, label):
    # Compute matches and mismatches
    matches = (output == label).sum()
    total_rows = len(output)
    match_percentage = (matches / total_rows) * 100

    print(f"Exact Match Percentage: {match_percentage:.2f}%")

def exact_match_metric_any(outputs, label):
    # Check if any entry in 'output' matches 'label' for each row
    matches = outputs.apply(lambda row: any([label[row.name] == val for val in row.values]), axis=1).sum()
    # print(matches)
    total_rows = len(outputs)
    match_percentage = (matches / total_rows) * 100

    print(f"Exact Match Percentage: {match_percentage:.2f}%")

def get_eval(label, predicted, possible_outputs, special_label):
    """
    Generate full evaulation of the model's output VS label
    """
    # Convert labels and predictions to binary based on special_label
    binary_label = [1 if lbl == special_label else 0 for lbl in label]
    binary_predicted = [1 if lbl == special_label else 0 for lbl in predicted]

    cm = confusion_matrix(binary_label, binary_predicted)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f'Not {special_label}', special_label])
    disp.plot()
    plt.show()

    accuracy = accuracy_score(binary_label, binary_predicted)
    precision = precision_score(binary_label, binary_predicted)
    recall = recall_score(binary_label, binary_predicted)

    print(pd.Series(binary_label).value_counts())
    print(pd.Series(binary_predicted).value_counts())
    print(f"Accuracy for {special_label} vs rest: {accuracy:.4f}")
    print(f"Precision for {special_label} vs rest: {precision:.4f}")
    print(f"Recall for {special_label} vs rest: {recall:.4f}")


In [6]:
# @title Load Json
import json

def load_json():
  with open('/content/drive/MyDrive/ProjectNLP/data/checkmate_in_one.json', 'r') as f:
      data = json.load(f)

  df = pd.DataFrame(data['examples'])
  df['clean_input'] = df['input']
  df['label'] = df['target']
  df['input'] = data['description'] + ' ' + df['input']#.replace('checkmate','check')
  check_test = df[['input', 'clean_input', 'label']]
  return check_test

In [7]:
# @title Filter
import re
def trim_move(output_df):
    pattern = re.compile(r'[A-Za-z]+\d+[#]')

    def trim(input):
        match = pattern.search(input)  # Search for the first occurrence of the pattern in the input
        if match:
            return match.group()  # Return the first matched substring
        return "garbage"

    return output_df.applymap(trim)


def get_first_valid_entry(row):
    for entry in row:
        if entry != "garbage":
            return entry
    return "garbage"

In [8]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

# **Tournament**

In [ ]:
# @title Load Model

model1 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/ProjectNLP/saved_models/original_chessgpt", low_cpu_mem_usage=True, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("Waterhorse/chessgpt-base-v1", padding_side='left', output_attentions=True)
tokenizer.pad_token = tokenizer.eos_token
model1.to('cuda:0')


In [ ]:
# @title llm_vs_engine_game
def random_player(board):
    """Selects a random legal move."""
    return random.choice(list(board.legal_moves))


def get_move_from_san(model, tokenizer, san_pos,fen_pos, engine):

    prompt = f"Q: Find the best move for black in the chess position fen: {fen_pos} {san_pos}"
    outputs = generate_outputs(model, tokenizer, batch=[prompt], beam_size=6, max_new_tokens=6, temp=0.7)
    moves = [output.strip().split()[0] for output in outputs[0]]
    #return best_move_from_candidates(san_pos,set(moves),engine )[0]
    move = sorted(moves, key=lambda x: moves.count(x), reverse=True)[0]

    print(moves)
    return move

def llm_vs_engine_game(model, tokenizer, engine_path):
    board = chess.Board()

    # Initialize the chess engine
    engine = chess.engine.SimpleEngine.popen_uci(engine_path)
    san_pos = ''
    while not board.is_game_over():
        if board.turn == chess.BLACK:
            # Model plays for white
            san_move = get_move_from_san(model, tokenizer, san_pos, board.fen(), engine)
            try:
              move = board.parse_san(san_move)
            except:
              move = random_player(board)
        else:
            # Engine plays for black
           result = engine.play(board, chess.engine.Limit(time=2, depth=1))
           move = result.move
           # move = random_player(board)
           san_move = board.san(move)

        if (len(board.move_stack) % 2 == 0):  # It's an even number, so start of a new move pair
            san_pos += f" {len(board.move_stack) // 2 + 1}. {san_move}"
        else:
            san_pos += f" {san_move}"

        board.push(move)

    engine.quit()
    print(board)
    print(san_pos)
    return board.result()

# Assuming you have already set up the model and tokenizer variables
result = llm_vs_engine_game(model1, tokenizer, maia_1100_path)  # replace with your engine path
print("Game Result:", result)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['e5', 'e5', 'e5', 'e5', 'e5', 'e5']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Nc6', 'Nc6', 'Nc6', 'Nc6', 'Nc6', 'Nc6']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

['Kf7', 'Kf7', 'bxa4', 'Kf6', 'Kf7', 'Kf7']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Ke6', '+-', 'Ke6', '+', 'Ke6', 'Kg7']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Kf6', 'Kf7', 'Kf7', 'Kf6', 'Kf6', 'Kf7']
. . . . . . . .
. . Q . . . . .
. p . . . . . .
k B . . . . . .
P . . Q P . . .
. . . P K . . .
. . . . . . . .
. . . . . . . .
 1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. d3 d6 5. Ng5 Nh6 6. Qf3 O-O 7. Nc3 Nd4 8. Qd1 Ng4 9. O-O h6 10. Nf3 Nxf3+ 11. Qxf3 Nf6 12. Qg3 Kh8 13. Nd5 Nxd5 14. Bxd5 c6 15. Bc4 f5 16. exf5 Bxf5 17. Be3 Bxe3 18. fxe3 Qg5 19. Qxg5 hxg5 20. e4 Bg6 21. Rxf8+ Rxf8 22. Rf1 Rxf1+ 23. Kxf1 A: 24. a4 Bf7 25. Bxf7 g6 26. Bxg6 Kg7 27. Bf5 Kf6 28. Bc8 b6 29. Bxa6 Ke7 30. Bb7 Kd7 31. b4 Kc7 32. Ba6 Kb8 33. b5 Kc7 34. bxc6 Kxc6 35. Bb5+ Kc5 36. c4 Kd4 37. Ke2 Ke3 38. Ke3 Kf4 39. g3 + 40. h4 gxh4 41. gxh4 Kg4 42. cxd5 b5 43. h5 Kxh5 44. h6 Kxh6 45. h7 Kxh7 46. h8=Q + 47. Qxe5 dxe5 48. d6+ Kg7 49. d7 Kf7 50. d8=Q Ke6 51. Qdd4+ Kf6 52. Qc7
Game Result: 1/2-1/2


In [ ]:
# @title Tournament
def tournament(model, tokenizer, engine_path, amount):
    results = {"LLM_Wins": 0, "Engine_Wins": 0, "Draws": 0}

    for i in range(amount):
        result = llm_vs_engine_game(model, tokenizer, engine_path)
        torch.cuda.empty_cache()

        gc.collect()
        if result == "0-1":  # Model (playing as Black) won
            results["LLM_Wins"] += 1
        elif result == "1-0":  # Engine (playing as White) won
            results["Engine_Wins"] += 1
        else:  # Draw
            results["Draws"] += 1

    return results

results = tournament(model1, tokenizer, maia_1100_path, 100)
print(results)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['e5', 'e5', 'e5', 'e5', 'e5', 'e5']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Nc6', 'Nc6', 'Nc6', 'Nc6', 'Nc6', 'Nc6']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

['Qf3+', 'Qf3+', 'Qf3+', 'Qf3+', 'Qf3+', 'Qf3+']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Qf1+', 'Qf1+', 'Qf1+', 'Qf1+', 'Qf1+', 'Qf1+']


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Qf3+', 'Qf3+', 'Qf3+', 'Qf3+', 'Qf3+', 'Qf3+']
. . . . . r k .
p p p . . p . .
. . . . . P p p
. . . . Q . . .
. . . . P . . .
. . . . . q . .
P P P . . . . R
. . . . . . R K
 1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. d3 Nf6 5. Ng5 O-O 6. O-O h6 7. Nf3 d6 8. Nc3 Bg4 9. h3 Bh5 10. g4 Bg6 11. Nh4 Bh7 12. Nf5 Bxf5 13. gxf5 Nd4 14. Be3 Nxf5 15. exf5 Bxe3 16. fxe3 d5 17. Bb3 Qd6 18. e4 dxe4 19. dxe4 Qc5+ 20. Rf2 Qe3 21. Nd5 Nxd5 22. Bxd5 Rad8 23. f6 Rxd5 24. Qxd5 Qg3+ 25. Rg2 Qe3+ 26. Kh2 Qf4+ 27. Kh1 Qf3 28. Rg1 g6 29. Qxe5 Qxh3+ 30. Rh2 Qf3+ 31. Rgg2 Qf1+ 32. Rg1 Qf3+ 33. Rgg2 Qf1+ 34. Rg1 Qf3+ 35. Rgg2 Qf1+ 36. Rg1 Qf3+ 37. Rgg2 Qf1+ 38. Rg1 Qf3+
{'LLM_Wins': 59, 'Engine_Wins': 8, 'Draws': 33}


# **Accuracy**

In [ ]:
# @title Generate Moves
def get_best_moves(positions, engine_path, analysis_time=4, depth=None):
    out = []
    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        for position in positions:
            board =load_SAN_position(position)
            # Get the best move for the resulting position
            if depth is not None:
              result = engine.play(board, chess.engine.Limit(time=analysis_time, depth = depth))
            else:
              result = engine.play(board, chess.engine.Limit(time=analysis_time))

            best_move = result.move

            out.append(str(board.san(best_move)))

    return out  # returning the move in SAN format

def load_SAN_position(san_pos):
    board = chess.Board()  # Start with a fresh board

    # Apply each move in the sequence to get the final position
    for move_san in san_pos.split():
        # Filter out move numbers
        if not move_san.endswith('.'):
            move = board.parse_san(move_san)
            board.push(move)
    return board

In [ ]:
adapters_name = "/content/drive/MyDrive/ProjectNLP/saved_models/lora_adapter_checkmate"
#model1, tokenizer = load_with_adapters(adapters_name)

tokenizer = AutoTokenizer.from_pretrained("Waterhorse/chessgpt-base-v1", padding_side='left', output_attentions=True)
tokenizer.pad_token = tokenizer.eos_token

model1 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/ProjectNLP/saved_models/model_lora_checkmate_new", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model1.to('cuda:0')

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)


In [ ]:
# @title Example
# EXPECT - MATE
prompt = '''Q:Is this san position 3k4/p3Q3/Pp6/1P5p/7P/6K1/8/2R1R3 a mate, a check or neither for White: A:'''
print(prompt)
print(f"EXPECT MATE {generate_outputs(model1, tokenizer, batch=[prompt], beam_size=12, max_new_tokens=1, temp=0.7)}")

# EXPECT - CHECK
prompt = '''Q:Is this san position 3k4/p3Q3/Pp6/1P5p/7P/6K1/8/1R2R3 a mate, a check or neither for White: A:'''
print(prompt)
print(f"EXPECT CHECK {generate_outputs(model1, tokenizer, batch=[prompt], beam_size=12, max_new_tokens=1, temp=0.7)}")

# EXPECT - NEITHER
prompt = '''Q:Is this san position 3k4/p4Q2/Pp6/1P5p/7P/6K1/8/1R2R3 a mate, a check or neither for White: A:'''
print(prompt)
print(f"EXPECT NEITHER {generate_outputs(model1, tokenizer, batch=[prompt], beam_size=12, max_new_tokens=1, temp=0.7)}")


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Q:Is this san position 3k4/p3Q3/Pp6/1P5p/7P/6K1/8/2R1R3 a mate, a check or neither for White: A:
EXPECT MATE {0: [' mate', ' mate', ' mate', ' mate', ' mate', ' mate', ' mate', ' mate', ' mate', ' mate', ' mate', ' mate']}
Q:Is this san position 3k4/p3Q3/Pp6/1P5p/7P/6K1/8/1R2R3 a mate, a check or neither for White: A:


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


EXPECT CHECK {0: [' check', ' check', ' check', ' check', ' check', ' check', ' check', ' check', ' check', ' check', ' check', ' check']}
Q:Is this san position 3k4/p4Q2/Pp6/1P5p/7P/6K1/8/1R2R3 a mate, a check or neither for White: A:
EXPECT NEITHER {0: [' neither', ' neither', ' neither', ' neither', ' neither', ' neither', ' neither', ' neither', ' neither', ' neither', ' neither', ' neither']}


In [ ]:
# @title Load Model

model1 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/ProjectNLP/saved_models/original_chessgpt", low_cpu_mem_usage=True, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("Waterhorse/chessgpt-base-v1", padding_side='left', output_attentions=True)
tokenizer.pad_token = tokenizer.eos_token
model1.to('cuda:0')


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)


In [ ]:
df = load_from_disk("/content/drive/MyDrive/ProjectNLP/data/final_datasets/rand_positions")['rand_positions_test'].to_pandas()
df

,input,clean_input
0,Q: Find the best move in the chess position 1....,1. Nf3 e6 2. Nc3 d5 3. a4 f5 4. h4 h5 5. Rh3 g...
1,Q: Find the best move in the chess position 1....,1. e4 c6 2. d4 d5 3. c4 dxe4 4. Nc3 Bf5 5. f3 ...
2,Q: Find the best move in the chess position 1....,1. e4 e5 2. Nf3 d6 3. Nc3 Be6 4. d3 c5 5. Be3 ...
3,Q: Find the best move in the chess position 1....,1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 4. Bxd7+ Nxd7 5...
4,Q: Find the best move in the chess position 1....,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. c3...
...,...,...
912,Q: Find the best move in the chess position 1....,1. e4 e5 2. Nf3 Nc6 3. d4 exd4 4. Nxd4 Nxd4 5....
913,Q: Find the best move in the chess position 1....,1. d4 e6 2. c4 d5 3. cxd5 exd5 4. Nc3 c6 5. g3...
914,Q: Find the best move in the chess position 1....,1. d4 e6 2. e3 b6 3. c4 Bb7 4. Nc3 h6 5. f3 g5...
915,Q: Find the best move in the chess position 1....,1. e4 e5 2. Nf3 Nc6 3. d4 exd4 4. Nxd4 Nxd4 5....


In [ ]:
results = evaluate_dataset(df['input'], model1, tokenizer, batch_size=2, max_new_tokens=8, n_examples=1000)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

[[' Nxd5 Qd6 10.', ' h5 Bh7 10. f4', ' Nxa8 Nc6 15.', ' e5 dxe5 10. d', ' Rfe1 Nh5 15.', ' Re1 Nh7 10. N', ' Nxh7 Kxh7', ' Qd2 Rae8 16.', ' dxc4 Qd4 15.', ' Qxf6+ Qf7 16', ' Nf3 Nc6 14.', ' Bb5 Ne4 16. B', ' Qc2 Rc8 11.', ' O-O-O Rc8', ' Re1 Bg4 12. h', ' Kd1 Qxf4 16.', ' Rfe1 Ne4 16. N', ' Qb3 Ne7 12. O', ' Ba4 b5 11. Bc', ' Qe2 a6 13. O', ' fxe4 e5 13. d', ' Rxe5 Nf4 13.', ' c5 Be7 10. Qc', ' Qxe4 Bg7 11.', ' Qxe5 Qe8 14.', ' Bxd5 Be6 11. B', ' dxc5 Nge7 10.', ' f4 Be6 12. Nd', ' Bxd3 O-O 14.', ' c4 Nc7 14. B', ' Rfe1 Rad8 16. Q', ' O-O Nb6 10.', ' c3 Nf6 10. Re', ' Nbd2 e6 10. Q', ' Bg5 h6 10. B', ' a4 Rfe8 14. N', ' Bxc6 Nxc6 15.', ' Qb4 Bxb4 12.', ' d3 Be7 10. Nbd', ' e4 Qf7 15. e', ' Qf4 Ncd4 11.', ' Nbd2 Qd7 10.', ' b4 Bb6 11. c', ' Rfe1 b4 16. N', ' fxe5 Bxb5 10.', ' b4 c6 15. Nxf', ' Kb1 e5 15. Be', ' Ne5 Nxe5 11. B', ' fxe5 Nxe5 12.', ' Qxf3 c6 15. B', ' Bxh7+ Kxh', ' Bxd5 c6 13. B', ' Ng5 h6 15. Nh', ' h4 Nd7 14. g', ' e4 c5 14. d5', ' Ne5 Qd6 10. B', ' Qc2 e5 14. N', ' Bf4 

In [ ]:
results.to_csv('rand_pos_results.csv', index = True)

In [ ]:
def trim_move(output_df):
    def trim(input):
        return input.split()[0]
    return output_df.applymap(trim)

#TRIM
output = results
output_columns = [col for col in results.columns if 'outputs_' in col]
output[output_columns] = trim_move(output[output_columns])
output = output.merge(df, on='input')
output.pop('input')

# Generate Engine moves for each position
output['MAIA 1100'] = get_best_moves(output['clean_input'], maia_1100_path, 2, 3)
output['MAIA 1400'] = get_best_moves(output['clean_input'], maia_1400_path,2)
output['MAIA 1600'] = get_best_moves(output['clean_input'], maia_1600_path,2)
output['MAIA 1900'] = get_best_moves(output['clean_input'], maia_1900_path,2)
output['Stockfish'] = get_best_moves(output['clean_input'], stockfish_path,2)

output

,outputs_0,outputs_1,outputs_2,outputs_3,outputs_4,clean_input,MAIA 1100,MAIA 1400,MAIA 1600,MAIA 1900,Stockfish
0,Nxd5,Nxd5,Rxe5,Nxd5,Nxd5,1. Nf3 e6 2. Nc3 d5 3. a4 f5 4. h4 h5 5. Rh3 g...,Nxd5,d4,Nxd5,Nxd5,d4
1,h5,h5,h5,h5,Qc2,1. e4 c6 2. d4 d5 3. c4 dxe4 4. Nc3 Bf5 5. f3 ...,h5,h5,h5,h5,h5
2,Nxa8,Nxa8,Nxa8,Nxa8,Nxa8,1. e4 e5 2. Nf3 d6 3. Nc3 Be6 4. d3 c5 5. Be3 ...,Nxa8,Nxa8,Nxa8,Nxa8,Nxa8
3,e5,e5,e5,e5,e5,1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 4. Bxd7+ Nxd7 5...,e5,e5,e5,e5,e5
4,Rfe1,Rac1,Rfe1,Rac1,Rfe1,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. c3...,h3,Rac1,Rac1,Rac1,Rac1
...,...,...,...,...,...,...,...,...,...,...,...
912,Nxe7+,Nxe7+,Nxe7+,Nxe7+,Nxe7+,1. e4 e5 2. Nf3 Nc6 3. d4 exd4 4. Nxd4 Nxd4 5....,Nxe7+,Nxe7+,Nxe7+,Nxe7+,Nxe7+
913,e4,e3,e3,e4,e3,1. d4 e6 2. c4 d5 3. cxd5 exd5 4. Nc3 c6 5. g3...,e4,e4,Re1,e4,e4
914,Kf2,Kf2,Kf2,Kf2,Kf2,1. d4 e6 2. e3 b6 3. c4 Bb7 4. Nc3 h6 5. f3 g5...,Kf2,Kf2,Kf2,Kf2,Kf2
915,Kd2,Kd1,Kd2,Kd1,Kd1,1. e4 e5 2. Nf3 Nc6 3. d4 exd4 4. Nxd4 Nxd4 5....,Kd2,Ke2,Kd1,Ke2,Ke2


In [ ]:
output.to_csv('/content/drive/MyDrive/ProjectNLP/rand_pos_moves_output.csv', index = True)

In [ ]:
MATE_SCORE = 32000


def winning_probability(cp):
    """
    Use a sigmoid or logistic function to map the engine score cp into [0 to 1] winning probability.
    Technically winning probability is like score rate because the game of chess has draw results.
    So winning probability is like (num_wins + num_draw/2)/total_games.

    cp is a score in centipawn unit.

    Ref: https://www.chessprogramming.org/Pawn_Advantage,_Win_Percentage,_and_Elo
    """
    # K is a factor to scale the winning probability, if K is low the winning probability is high.
    # Stronger engine generally has a lower K value that is if stronger engine has an advantage of
    # 75 centipawn, while a weaker engine has an advantage of 75 centipawn too, it is the stronger
    # engine that has a higher chance to win the game.
    K = 4

    p = cp/100  # convert centipawn to pawn unit

    return 1 / (1 + 10 ** (-p/K))

# FROM WHITE PRESPECTIVE
def winrate_position(fen, engine, limit):
    board = chess.Board(fen)
    with engine.analysis(board, limit=chess.engine.Limit(time=limit)) as analysis:
        for info in analysis:
            eng_score = info.get("score")
            if eng_score is not None:
                if board.turn == chess.BLACK:
                    score = -eng_score.relative.score(mate_score=MATE_SCORE)
                else:
                    score = eng_score.relative.score(mate_score=MATE_SCORE)
        raise ValueError("Unable to determine score for position.")

def eval_moves(position, moves, engine_path):
    """
    Given a position and multiple moves, output the accuracies of the moves
    """
    scores = {}
    engine = chess.engine.SimpleEngine.popen_uci(engine_path)

    # Remove move numbers
    cleaned_position = re.sub(r'\d+\.', '', position).strip()

    # Split into individual SAN moves and remove any annotations like '?'
    moves_san = cleaned_position.split()
    moves_san = [re.sub(r'[?!]', '', move) for move in moves_san]

    board = chess.Board()
    for move_san in moves_san:
        move = board.parse_san(move_san)
        board.push(move)

    base_winchance = winrate_position(board.fen(), engine, 2)

    for key, move in moves.items():
        clean_move = re.sub(r'[?!]', '', move)
        try:
          move_obj = board.parse_san(clean_move)
          board.push(move_obj)

          post_move_winrate = winrate_position(board.fen(), engine, 2)
          board.pop()  # Return to the original position to evaluate the next move

          if post_move_winrate is None:
              scores[key] = float('nan')
              continue

          scores[key] = post_move_winrate - base_winchance

        except:
          scores[key] = MATE_SCORE - base_winchance

    engine.quit()
    return pd.Series(scores)


In [ ]:
# Calcuate accuracy using Lichess metric
maia_columns = ['MAIA 1100', 'MAIA 1400', 'MAIA 1600', 'MAIA 1900', 'Stockfish']

to_eval = output_columns + maia_columns#['MAIA 1100', 'MAIA 1400', 'MAIA 1600', 'MAIA 1900']
accuracies = output.apply(lambda row: eval_moves(row['clean_input'],row[to_eval] , stockfish_path), axis=1)
accuracies['top output'] = accuracies[output_columns].apply(max,axis=1)


In [ ]:
output.to_csv('/content/drive/MyDrive/ProjectNLP/accuracies.csv', index = True)

In [ ]:
to_eval.append('top output')
to_eval = to_eval + ['Stockfish']
for col in to_eval:
  print(f"Centipawn for {col} : {accuracies[col].mean()}")

for col in to_eval:
  print(f"Centipawn for {col} : {accuracies[col].mean()}")

# Calcuate similarity of ChessGPT move with the engines
accuracy_similarity = {}
for engine_col in maia_columns + ['Stockfish']:
    matches = (output['outputs_0'] == output[engine_col]).sum()
    total_rows = len(output)
    print(f'Similarity of ChessGPT with {engine_col}: {matches / total_rows} ')


Accuracy for outputs_0 : -100.69901853871319
Accuracy for outputs_1 : -125.82442748091603
Accuracy for outputs_2 : -250.89640130861505
Accuracy for outputs_3 : -258.6324972737186
Accuracy for outputs_4 : -122.81243184296619
Accuracy for MAIA 1100 : -30.115594329334787
Accuracy for MAIA 1400 : -13.86041439476554
Accuracy for MAIA 1600 : -11.007633587786259
Accuracy for MAIA 1900 : -6.14721919302072
Accuracy for Stockfish : -9.174482006543075
Accuracy for top output : 31.479825517993458
Accuracy for Stockfish : -9.174482006543075
Similarity of ChessGPT with MAIA 1100: 0.5398037077426391 
Similarity of ChessGPT with MAIA 1400: 0.44711014176663033 
Similarity of ChessGPT with MAIA 1600: 0.44492911668484186 
Similarity of ChessGPT with MAIA 1900: 0.4525627044711014 
Similarity of ChessGPT with Stockfish: 0.38495092693565974 
Similarity of ChessGPT with Stockfish: 0.38495092693565974 


In [ ]:
# TEST
# EXPECT - MATE
prompt = '''Q:Is this san position 7R/4kr2/1pq2Q2/6P1/p3p3/P5P1/2P1BK2/3R4 w a mate, a check or neither for White: A:'''
print(prompt)
print(f"EXPECT MATE {generate_outputs(model1, tokenizer, batch=[prompt], beam_size=12, max_new_tokens=1, temp=0.7)}")

# EXPECT - CHECK
prompt = '''Q:Is this san position 7R/4kr2/1pq2Q2/6P1/p3p3/P5P1/2P1BK2/R7 a mate, a check or neither for White: A:'''
print(prompt)
print(f"EXPECT CHECK {generate_outputs(model1, tokenizer, batch=[prompt], beam_size=12, max_new_tokens=1, temp=0.7)}")

# EXPECT - NEITHER
prompt = '''Q:Is this san position 7R/4kr2/1pq5/5QP1/p3p3/P5P1/2P1BK2/R7 a mate, a check or neither for White: A:'''
print(prompt)
print(f"EXPECT NEITHER {generate_outputs(model1, tokenizer, batch=[prompt], beam_size=12, max_new_tokens=1, temp=0.7)}")


In [ ]:
accuracies.to_csv('/content/drive/MyDrive/ProjectNLP/rand_pos_moves_output.csv', index = True)

# **Connection Helpers**

In [ ]:
import io
import chess.engine, chess.pgn, chess

# extract only the move from a single string output
def mapMove(input):
    return input.split('... ')[-1].split()[0]

# extract the moves from the outputs
def extractMove(df):
    cols = [col for col in df.columns if 'outputs_' in col]
    df[cols] = df[cols].applymap(mapMove)
    print(df)
    return df

# get a state of the board from SAN code
def get_board_from_san(san):
    pgn = io.StringIO(san)
    game = chess.pgn.read_game(pgn)
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
    return board

# check if move is legal
def isLegalMove(san, move):
    board = get_board_from_san(san)

    # After all moves have been played, check the next move's legality
    legal_moves = [board.san(m) for m in board.legal_moves]
    if move in legal_moves:
        return 1
    else:
        return 0

# get SAN from prompt
def getSan(prompt):
    try:
      return prompt.split('Find a move in the chess position resulting in checkmate ')[1].split(' {Now')[0]
    except:
      return prompt.split('Find a move in the chess position resulting in checkmate ')[1]

# convert all input prompts to SANs
def inputToSan(df):
    cols = [col for col in df.columns if 'input' in col]
    df[cols] = df[cols].applymap(getSan)
    print(df)
    return df

# create legals_ columns with the output move added to SAN (# and + trimmed)
def legalSans(df):
    cols = [col for col in df.columns if 'outputs_' in col]
    sans = df['input']
    k=0
    for column in cols:
        legals = []
        for i, row in df.iterrows():
            legals.append(sans[i] + ' ' + row[column].rstrip('#').rstrip('+'))
        df[f'legals_{k}'] = legals
        k += 1
    print(df)
    return df

# create a board dataframe with all output columns converted into boards
def sanBoards(df):
    cols = [col for col in df.columns if 'outputs_' in col]
    boards = df
    boards = boards[cols].applymap(get_board_from_san)
    print(boards)
    return boards

# convert SAN to FEN (mirroring the board where needed)
def getFEN(san):
    pgn = io.StringIO(san)
    game = chess.pgn.read_game(pgn)
    while game.next():
        game=game.next()
        # print(mygame.board().fen())
    print(game.board().fen().split(' '))
    res = game.board().fen()
    if game.board().fen().split(' ')[1] == 'w':
        res = game.board().mirror().fen()
    return res

# create new dataframe (fen) with legals_ columns storing now FEN format instead of PGN / SAN
def pgnToFen(df):
    cols = [col for col in df.columns if 'legals_' in col]
    print(cols)
    fen = df
    fen = fen[cols].applymap(getFEN)
    print(fen)
    return fen

# defining final answer based on first # appearance in the output of first model
def outputFun(df):
    cols = [col for col in df.columns if 'outputs_' in col]
    final = [None] * len(df['input'])
    for i, row in df.iterrows():
        for column in cols:
            if '#' in row[column]:
                final[i] = row[column]
                break
    df['final_output'] = final
    print(df)
    return df

# compare all/multiple outputs with the true label
def exact_match_metric_any(outputs, label):
    # Check if any entry in 'output' matches 'label' for each row
    matches = outputs.apply(lambda row: any([label[row.name] == val for val in row.values]), axis=1).sum()
    # print(matches)
    total_rows = len(outputs)
    match_percentage = (matches / total_rows) * 100
    print(f"Exact Match Percentage: {match_percentage:.2f}%")

# create a prompt for the second model based on the FEN
def addPrompt(fen):
  prompt = f'Q: Is this san position {fen} a mate, a check or neither for White: A:'
  return prompt

# prepare a dataframe with the prompts for the second model
def prepareM2(df):
  cols = [col for col in df.columns if 'legals_' in col]
  df = df[cols].applymap(addPrompt)
  return df

# out of the 2nd model outputs choose the answer (check, mate, neither) with highest frequency
def finalOut(results):
    final = []
    for i, row in results.iterrows():
        counts = row.value_counts()
        final.append(counts.index[0])
    return final

# based on the 2nd model majority answer for each output of the 1st model,
# count the output with highest frequency of 'mate' answer
# strip + and add # at the end
def finalAns(df):
    cols = [col for col in df.columns if 'final_legals_' in col]
    finals = [None]*len(df['input'])
    for i, row in df.iterrows():
        temp = {}
        for column in cols:
            ind = column.split('_')[2]
            if 'mate' in row[column]:
                if row['outputs_'+ind] in temp:
                    temp[row['outputs_'+ind]] += 1
                else:
                    temp[row['outputs_'+ind]] = 1
        if not any(temp):
            finals[i] = 'None'
        else:
            clean = max(zip(temp.values(), temp.keys()))[1].rstrip('#').rstrip('+') +'#'
            finals[i] = clean
        # print(finals)
    df['finals'] = finals
    return df

def filter(output):
    cols = [col for col in output.columns if 'outputs_' in col]
    final = []
    for i, row in output.iterrows():
        counts = row[cols].value_counts()
        final.append(int(len(counts)))
    output['counts'] = final
    output = output.loc[output['counts']>1]
    return output


# **Complete Evaluation**
Runs all the steps: prompt/s -> model1 -> boards -> model2 -> final answer --> accuracy evaluation

### **PART 1: (PGN Prompt, Label) -> ChessGPT --> n moves**

**Load first model (ChessGPT)**

In [ ]:
# empty cash

import gc
import torch
gc.collect()
torch.cuda.empty_cache()

# load model #1

tokenizer = AutoTokenizer.from_pretrained("Waterhorse/chessgpt-base-v1", padding_side='left', output_attentions=True)
tokenizer.pad_token = tokenizer.eos_token

model1 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/ProjectNLP/saved_models/original_chessgpt", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model1.to('cuda:0')

**Load data: PGN prompts and Labels**

In [ ]:
# empty cash

import gc
import torch
gc.collect()
torch.cuda.empty_cache()

# load test data

df = load_json()

**Evaluate ChessGPT results and Add moves**

In [ ]:
# empty cash

gc.collect()
torch.cuda.empty_cache()

# run evaluation of the first model output



output = evaluate_dataset(df['input'],model1, tokenizer, batch_size=1,beam_size=6, max_new_tokens=6, n_examples=3500,label_data_in= df['label'])
output_columns = [col for col in output.columns if 'outputs_' in col]
trim_output = trim_move(output[output_columns])
trim_output['final_output'] = trim_output[output_columns].apply(get_first_valid_entry, axis=1)
print(trim_output)



In [ ]:
print('\n Roys version \n')

evaluation_result = exact_match_metric(trim_output['final_output'], output['label'])
evaluation_result = exact_match_metric( trim_output['outputs_0'], output['label'])
evaluation_result = exact_match_metric_any(trim_output[output_columns], output['label'])
evaluation_result = exact_match_metric_any(trim_output[output_columns[:4]], output['label'])
#print(output[['final_output','label']])

print('\n Ksenias version \n')

# get san out of input
df1 = inputToSan(output)
# perform the moves based on each output
df1 = extractMove(df1)
# concatinate move to san (while trimming # or +) and save it under "legals_" columns
df1 = legalSans(df1)


df1 = outputFun(df1)
cols = [col for col in df1.columns if 'outputs_' in col]
# Evaluate accuracy of the final output based on the first # checkmate sign
exact_match_metric(df1['final_output'], df1['label'])
# Evaluate accuracy of the final output based on all outputs
exact_match_metric_any(df1[cols], output['label'])

**Save 1st model output**

In [ ]:
# save 1st model output

output.to_csv('output_model1.csv', index = True)
trim_output.to_csv('output_model1_Roy.csv', index = True)
df1.to_csv('output_model1_Ksenia.csv', index = True)

### **PART2: (FEN Board, n moves) -> n FEN boards**

**Convert initial PGN to FEN board**

In [ ]:
# create FENs for each san
fens = pgnToFen(df1)


### **PART3: Boards -> 2nd model -> m Answers for each board**

**Load the 2nd model**

In [ ]:
# empty cash

torch.cuda.empty_cache()
gc.collect()

# load second model

tokenizer = AutoTokenizer.from_pretrained("Waterhorse/chessgpt-base-v1", padding_side='left', output_attentions=True)
tokenizer.pad_token = tokenizer.eos_token

model2 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/ProjectNLP/saved_models/model_lora_checkmate_new", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model2.to('cuda:0')

**Check 2nd model on the test sample**

In [ ]:
torch.cuda.empty_cache()
gc.collect()

dataset = load_from_disk("/content/drive/MyDrive/ProjectNLP/data/final_datasets/classify_into_checkmate_check_none_test")
test_datasets = {k: v for k, v in dataset.items() if k.endswith('_test')}

# Run test
torch.cuda.empty_cache()
gc.collect()
test_datasets['fen_mate_in_one_test'] = test_datasets['fen_mate_in_one_test'].shuffle()
test = test_datasets['fen_mate_in_one_test'].to_pandas()
print(len(test['input']))
output_test = evaluate_dataset(test['input'], model2, tokenizer, batch_size=1,beam_size=6, max_new_tokens=1, n_examples=2500,label_data_in= test['label'])


**Display the results of the test**

In [ ]:
torch.cuda.empty_cache()
gc.collect()

outputs_columns = [col for col in output_test.columns if col.startswith('outputs_')]

# Define the trimming function
def trim_for_bool(value):
    return value.strip()

# Specify the columns to trim
columns_to_trim = [col for col in output_test.columns if col.startswith('outputs_')]

# Apply the trimming function to the specified columns
output_test[columns_to_trim] = output_test[columns_to_trim].applymap(trim_for_bool)

# Evaluate the most common output against the label
evaluation_result = get_eval(output_test['label'], output_test['outputs_1'], ['neither', 'check', 'mate'], 'mate')
print(evaluation_result)

**Run the 2nd model**

In [ ]:
# ask the 2nd model if the FEN of ea. board is check/checkmate/neither
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
# convert FEN boards into Prompts needed for the second model (including mirroring the board when needed)
fens = prepareM2(fens)
print(fens)
fens.to_csv('test.csv', index = True)


**Choose the final result (mate, check, neither) by Majority vote for every output**

In [ ]:
cols = [col for col in fens.columns if col.startswith('legals_')]
for column in cols:
  torch.cuda.empty_cache()
  gc.collect()
  size = len(fens['legals_1'])
  results = evaluate_dataset(fens[column], model2, tokenizer, batch_size=1, max_new_tokens=1, n_examples=size)
  torch.cuda.empty_cache()
  gc.collect()
  name = 'final_' + column
  df1[name] = finalOut(results)
print(df1)

df1.to_csv('output_model2_major.csv', index = True)

### **PART4: Multiple answers -> Heuristics -> Single final answer**

**Choose the best output by majority vote**

In [ ]:
# apply majority vote to create a final output
output = finalAns(df1)
# print(df1)
output.to_csv('output_model2_finals.csv', index = True)

### **PART5: Accuracy Evaluation**

**Evaluate accuracy**

In [ ]:
cols = [col for col in output.columns if col.startswith('outputs_')]
print('\n ChatGPT accuracy (based on output_0)')
exact_match_metric(output['outputs_0'], output['label'])
print('\n ChatGPT accuracy (based on final_output)')
exact_match_metric(output['final_output'], output['label'])
print('\n ChatGPT accuracy based on any of 5 outputs')
exact_match_metric_any(output[cols], output['label'])
print('\n Our accuracy following the 2nd model run')
exact_match_metric(output['finals'], output['label'])

output_filtered = filter(output)
print("\n Number of non-distinct lines ")
print(len(output_filtered['outputs_0']))
print('\n ChatGPT accuracy (based on output_0) for the lines which did not have disctinct answer')
exact_match_metric(output_filtered['outputs_0'], output_filtered['label'])
print('\n ChatGPT accuracy (based on final_output) for the lines which did not have disctinct answer')
exact_match_metric(output_filtered['final_output'], output_filtered['label'])
print('\n ChatGPT accuracy (based on any of 5 outputs) for the lines which did not have disctinct answer')
exact_match_metric_any(output_filtered[cols], output_filtered['label'])
print('\n Our accuracy for the lines which did not have disctinct answer')
exact_match_metric(output_filtered['finals'], output_filtered['label'])
